In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.fft import fft2, ifft2, fftshift, ifftshift

#Load and add multiple periodic noises ---
image = cv2.imread('Lena.jpg', 0)
rows, cols = image.shape
F = fftshift(fft2(image))

max_val = np.max(np.abs(F))
amp = 0.05 * max_val  # amplitude of each noise spike

# Noise source 1 (low frequency diagonal)
u_offset1, v_offset1 = 50, 30
F[rows//2 + v_offset1, cols//2 + u_offset1] += amp
F[rows//2 - v_offset1, cols//2 - u_offset1] += amp

# Noise source 2 (different direction/frequency)
u_offset2, v_offset2 = 70, -40
F[rows//2 + v_offset2, cols//2 + u_offset2] += amp
F[rows//2 - v_offset2, cols//2 - u_offset2] += amp

# Inverse FFT → noisy image
noisy_image = np.abs(ifft2(ifftshift(F)))

# Compute log magnitude spectrum 
magnitude_spectrum = np.log1p(np.abs(F))

# Mask DC region
mask = np.ones_like(magnitude_spectrum)
cv2.circle(mask, (cols//2, rows//2), 10, 0, -1)
spectrum_masked = magnitude_spectrum * mask

# --- Step 4: Threshold + NMS to detect spikes automatically ---
mean_val = np.mean(spectrum_masked)
std_val = np.std(spectrum_masked)
T = mean_val + 3 * std_val
binary_mask = (spectrum_masked > T).astype(np.uint8)

# Normalize for NMS
norm = cv2.normalize(spectrum_masked, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
kernel = np.ones((21,21), np.uint8)
local_max = cv2.dilate(norm, kernel)
nms = np.zeros_like(norm, np.uint8)
nms[(norm == local_max) & (binary_mask == 1)] = 255

coords = cv2.findNonZero(nms)
if coords is None:
    raise ValueError("No spikes detected!")
coords = [tuple(pt[0]) for pt in coords]

# --- Step 5: Build Butterworth Notch Reject Filter ---
def butterworth(img,uk_list, vk_list, D0, n):
    M, N = img.shape
    H = np.ones((M, N), dtype=np.float32)
    for k in range(len(uk_list)):
        uk = uk_list[k]
        vk = vk_list[k]
        for u in range(M):
            for v in range(N):
                Dk = np.sqrt((u - N/2 - uk)**2 + (v - M/2 - vk)**2)
                #D_k = np.sqrt((u - N/2 + uk)**2 + (v - M/2 + vk)**2)

                if Dk == 0:
                    Dk = 1e-6
                #if D_k == 0:
                    #D_k = 1e-6

                # Butterworth notch reject formula
                Hk = 1 / (1 + (D0 / Dk)**(2 * n))
               #H_k = 1 / (1 + (D0 / D_k)**(2 * n))
                H[v, u] = H[v, u] * Hk


    return H

# --- Step 6: Apply Butterworth notch filter ---
D0 = 10
n = 2
uk_list = []
vk_list = []

for point in coords:
    t1 = 256 - point[0]
    t2 = 256 - point[1]
    uk_list.append(t1)
    vk_list.append(t2)
H = butterworth(image, uk_list,vk_list,D0,n)
F_filtered = F * H

# --- Step 7: Inverse FFT to reconstruct image ---
recovered_image = np.abs(ifft2(ifftshift(F_filtered)))

# --- Step 8: Visualization ---
plt.figure(figsize=(15,9))
plt.subplot(2,3,1), plt.imshow(image, cmap='gray'), plt.title('Original')
plt.subplot(2,3,2), plt.imshow(noisy_image, cmap='gray'), plt.title('Noisy Image (2 periodic noises)')
plt.subplot(2,3,3), plt.imshow(np.log1p(np.abs(F)), cmap='gray'), plt.title('FFT Magnitude (with multiple spikes)')
plt.subplot(2,3,4), plt.imshow(binary_mask*255, cmap='gray'), plt.title('Thresholded Peaks')
plt.subplot(2,3,5), plt.imshow(H, cmap='gray'), plt.title('Butterworth Notch Filter')
plt.subplot(2,3,6), plt.imshow(recovered_image, cmap='gray'), plt.title('Recovered Image')
plt.tight_layout()
plt.show()

cv2.imwrite('im2.png', noisy_image)

print("Detected spike coordinates:", coords)


SyntaxError: invalid syntax (7696906.py, line 6)